In [1]:
import pickle
import os
import pandas as pd
import toml

In [2]:
if os.name == "posix":  # Linux, macOS
    import mysql.connector  #MySQL en Linux, macOS
elif os.name == "nt":  # Windows
    import pymysql  #PyMySQL en Windows
else:
    raise Exception("Sistema operativo no soportado")

config = toml.load("../.streamlit/secrets.toml")
db_config = config["database"]
database = db_config["database"]

# Conectar a MySQL
if os.name == "posix":  # Linux/macOS
    conn = mysql.connector.connect(
        host=db_config["host"],
        user=db_config["user"],
        password=db_config["password"],
        database=database
    )
elif os.name == "nt":  # Windows
    conn = pymysql.connect(
        host=db_config["host"],
        user=db_config["user"],
        password=db_config["password"],
        database=database,
        cursorclass=pymysql.cursors.DictCursor)

cursor = conn.cursor()

# Consulta obtener todas las tecnologías
query_tecnologias = "SELECT * FROM tecnologias;"
df_tecnologias_original = pd.read_sql(query_tecnologias, conn)

# Consulta obtener la relación entre ofertas y tecnologías
query_tecnologias_relacion = "SELECT * FROM tecnologias_relacion;"
df_tecnologias_relacion_original = pd.read_sql(query_tecnologias_relacion, conn)

# Consulta obtener todas las habilidades
query_habilidades = "SELECT * FROM habilidades;"
df_habilidades_original = pd.read_sql(query_habilidades, conn)

# Consulta obtener la relación entre ofertas y habilidades
query_habilidades_relacion = "SELECT * FROM habilidades_relacion;"
df_habilidades_relacion_original = pd.read_sql(query_habilidades_relacion, conn)

# Cerrar la conexión
cursor.close()
conn.close()

/tmp/ipykernel_21736/3931662679.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tecnologias_original = pd.read_sql(query_tecnologias, conn)
/tmp/ipykernel_21736/3931662679.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tecnologias_relacion_original = pd.read_sql(query_tecnologias_relacion, conn)
/tmp/ipykernel_21736/3931662679.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_habilidades_original = pd.read_sql(query_habilidades, conn)
/tmp/ipykernel_21736/3931662679.py:44: UserWarning: pandas only sup

In [3]:
manfred_habilidades_matricial = pd.read_pickle("Pickles/manfred_habilidades_matricial_actualizacion.pkl")
manfred_tecnologias_matricial = pd.read_pickle("Pickles/manfred_programas_matricial_actualizacion.pkl")
tecnoempleo_habilidades_matricial = pd.read_pickle("Pickles/habilidades_matricial_tecnoempleo_actualizacion.pkl")
tecnoempleo_tecnologias_matricial = pd.read_pickle("Pickles/tecnologias_matricial_tecnoempleo_actualizacion.pkl")
df_tecnologias_matricial_actualizacion = pd.concat([tecnoempleo_tecnologias_matricial, manfred_tecnologias_matricial], ignore_index=True).fillna(0)
df_habilidades_matricial_actualizacion = pd.concat([tecnoempleo_habilidades_matricial, manfred_habilidades_matricial], ignore_index=True).fillna(0)

/tmp/ipykernel_21736/3859967669.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_tecnologias_matricial_actualizacion = pd.concat([tecnoempleo_tecnologias_matricial, manfred_tecnologias_matricial], ignore_index=True).fillna(0)


In [4]:
def procesar_tecnologias():
        
    # Transformar el DataFrame de formato ancho a largo
    df_largo = df_tecnologias_matricial_actualizacion.melt(id_vars=["id"], var_name="tecnologia", value_name="presente")
            
    # Filtrar solo los valores donde "presente" es 1
    df_largo = df_largo[df_largo["presente"] == 1].drop(columns=["presente"])
            
    # Serie solo con las tecnologías de la nueva extracción
    df_tecnologias_nueva_extraccion = pd.DataFrame({"tecnologia": df_largo["tecnologia"].unique()}) 
    
    # Identificar tecnologías nuevas a partir del df tecnologias ya existente
    tecnologias_nuevas = df_tecnologias_nueva_extraccion[~df_tecnologias_nueva_extraccion["tecnologia"].isin(df_tecnologias_original["tecnologia"])]
    #aqui necesito la lista completa de tecnologias
    ultimo_id_tecnologias = df_tecnologias_original["tec_id"].max()
    #aqui el ultimo id de tecnologias
    tecnologias_nuevas["tec_id"] = [ultimo_id_tecnologias + i + 1 for i in range(len(tecnologias_nuevas))]
    
    # Concatenar las nuevas tecnologías con el DataFrame original
    df_tecnologias_completo = pd.concat([df_tecnologias_original, tecnologias_nuevas[["tec_id", "tecnologia"]]], ignore_index=True)
    df_tecnologias_completo["tec_id"] = df_tecnologias_completo["tec_id"].astype(int)
    
    # Relacionar los IDs de tecnologías con los IDs originales
    
    df_relacion_tecnologias = df_largo.merge(df_tecnologias_completo, on="tecnologia", how="left").drop(columns=["tecnologia"])
    
    
    # Guardar los resultados en archivos CSV
    tecnologias_nuevas.to_pickle("Pickles/nuevas_tecnologias.pkl")
    df_relacion_tecnologias.to_pickle("Pickles/nuevas_relacion_tecnologias.pkl")
    
    return df_tecnologias_completo, df_relacion_tecnologias

In [5]:
procesar_tecnologias()

/tmp/ipykernel_21736/2974558336.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tecnologias_nuevas["tec_id"] = [ultimo_id_tecnologias + i + 1 for i in range(len(tecnologias_nuevas))]


(     tecnologia  tec_id
 0        Python       1
 1           C++       2
 2     Cinema 4D       3
 3           CRM       4
 4         Canva       5
 ...         ...     ...
 2117    AWS EC2    2118
 2118   Solidity    2119
 2119       Scss    2120
 2120      Remix    2121
 2121      Astro    2122
 
 [2122 rows x 2 columns],
                id  tec_id
 0     id_tec_3781    1705
 1     id_tec_4014     590
 2     id_tec_4248    1287
 3     id_tec_4284    1287
 4     id_tec_3693     445
 ...           ...     ...
 2538  id_man_1246      61
 2539  id_man_1247      69
 2540  id_man_1247      83
 2541  id_man_1247    2122
 2542  id_man_1247     274
 
 [2543 rows x 2 columns])

In [6]:
def procesar_habilidades():    
    
    # Transformar el DataFrame de formato ancho a largo
    df_largo = df_habilidades_matricial_actualizacion.melt(id_vars=["id"], var_name="habilidad", value_name="presente")
    
    # Filtrar solo los valores donde "presente" es 1
    df_largo = df_largo[df_largo["presente"] == 1].drop(columns=["presente"])
    
    # Serie solo con las habilidades de la nueva extracción
    df_habilidades_nueva_extraccion = pd.DataFrame({"habilidad": df_largo["habilidad"].unique()}) 
    
    # Identificar habilidades nuevas a partir del df tecnologias ya existente
    habilidades_nuevas = df_habilidades_nueva_extraccion[~df_habilidades_nueva_extraccion["habilidad"].isin(df_habilidades_original["habilidad"])]
    
    ultimo_id_habilidades = df_habilidades_original["hab_id"].max()
    habilidades_nuevas["hab_id"] = [ultimo_id_habilidades + i + 1 for i in range(len(habilidades_nuevas))]
    
    # Concatenar las nuevas habilidades con el DataFrame original
    df_habilidades_completo = pd.concat([df_habilidades_original, habilidades_nuevas[["hab_id", "habilidad"]]], ignore_index=True)
    df_habilidades_completo
    
    df_habilidades_completo["hab_id"] = df_habilidades_completo["hab_id"].astype(int)
    
    # Relacionar los IDs de habilidades con los IDs originales
    df_relacion_habilidades = df_largo.merge(df_habilidades_completo, on="habilidad", how="left").drop(columns=["habilidad"])
    
    habilidades_nuevas.to_pickle("Pickles/nuevas_habilidades.pkl")
    df_relacion_habilidades.to_pickle("Pickles/nuevas_relacion_habilidades.pkl")
    
    return df_habilidades_completo, df_relacion_habilidades

In [7]:
procesar_habilidades()

(                        habilidad  hab_id
 0            Pensamiento creativo       1
 1   Tolerancia a la incertidumbre       2
 2    Resistencia a la frustración       3
 3             Comunicación verbal       4
 4            Capacidad de escucha       5
 ..                            ...     ...
 60                 Administrativo      61
 61                 Diseño gráfico      62
 62                 Arquitecto TIC      63
 63               Técnico Software      64
 64                         DevOps      65
 
 [65 rows x 2 columns],
                id  hab_id
 0     id_tec_3764      48
 1     id_tec_3815      48
 2     id_tec_3925      48
 3     id_tec_3951      48
 4     id_tec_3967      48
 ...           ...     ...
 1103  id_man_1242      24
 1104  id_man_1244      24
 1105  id_man_1245      24
 1106  id_man_1246      24
 1107  id_man_1247      24
 
 [1108 rows x 2 columns])